In [1]:
!pip install transformers
!pip install pytorch-pretrained-bert
!pip install datasets
!pip install wandb
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 96.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 920.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3

In [2]:
from transformers import BertModel, BertTokenizerFast, AdamW
import json
import torch
from tqdm import tqdm
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from torch.optim.lr_scheduler import ExponentialLR
from torch import nn
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import default_data_collator
from transformers import BertModel
import wandb
import numpy as np
import evaluate
import collections


# Config

In [ ]:
wandb.init(project="bert-qa", entity="igorpestretsov")
wandb.config.update({
  "learning_rate": 2e-5,
  "epochs": 3,
  "batch_size": 64,
  "scheduler_gamma":0.9,
  "optim_weight_decay":2e-2,
  })  

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Data preprocessing

In [3]:
model_checkpoint ="bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
bert = BertModel.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
raw_datasets = load_dataset("squad_v2")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:

def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context" ],
        truncation="only_second" ,
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
train_dataset = raw_datasets["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

  0%|          | 0/131 [00:00<?, ?ba/s]

(130319, 132079)

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" ],
        examples["context" ],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

  0%|          | 0/12 [00:00<?, ?ba/s]

(11873, 12199)

In [ ]:
train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=wandb.config['batch_size'],
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=wandb.config['batch_size'],
)

# loss and metrics


In [ ]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    loss_fct = nn.CrossEntropyLoss()
    start_loss = loss_fct(start_logits, start_positions)
    end_loss = loss_fct(end_logits, end_positions)
    total_loss = (start_loss + end_loss)/2
    return total_loss

In [ ]:
n_best = 20
max_answer_length = 30

metric = evaluate.load("squad_v2")

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(all_start_logits, all_end_logits, features, examples):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()]
    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    metrics = metric.compute(predictions=formatted_predictions, references=theoretical_answers)
    print(metrics)
    return metrics


# train and eval func

In [ ]:
def train_epoch(model, dataloader, epoch):
    model = model.train()
    losses = []
    acc = []
    ctr = 0

    for step, batch in tqdm(enumerate(dataloader), desc = f'Epoch {epoch} ',total=len(dataloader)):

        optim.zero_grad()
        batch = {k: batch[k].to(device) for k in batch}
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        token_type_ids = batch['token_type_ids']
        start_positions = batch['start_positions']
        end_positions = batch['end_positions']

        out_start, out_end = model(input_ids=input_ids, 
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)
        loss = loss_fn(out_start, out_end, start_positions, end_positions) 
        loss.backward()
        optim.step()
        
        start_pred = torch.argmax(out_start, dim=1)
        end_pred = torch.argmax(out_end, dim=1)
        acc.append(((start_pred == start_positions).sum()/len(start_pred)).item())
        acc.append(((end_pred == end_positions).sum()/len(end_pred)).item())
        batch_acc = sum(acc)/len(acc)
        wandb.log({"EM": batch_acc, "loss":loss.item(), "lr":optim.param_groups[0]['lr']})
        if step % 100==0:
          print(f' {step}/{len(dataloader)} EM:{batch_acc:.2f} | Loss:{loss.item():.2f} | lr:{optim.param_groups[0]["lr"]:.10f}')
        if step % 300 ==0:
          scheduler.step()


In [ ]:
def eval_model(model, dataloader):
    model.eval()
    start_logits = []
    end_logits = []
    for batch in tqdm(dataloader):
        with torch.no_grad():
            batch = {k: batch[k].to(device) for k in batch}
            outputs = model(**batch)

        start_logits.append(outputs[0].cpu().numpy())
        end_logits.append(outputs[1].cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = postprocess_qa_predictions(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    wandb.log(metrics)

#model arch

In [6]:
class QAModel(nn.Module):
    def __init__(self):
        super(QAModel, self).__init__()
        self.bert = bert_model
        self.drop_out = nn.Dropout(0.1)
        self.l1 = nn.Linear(768, 2)
        self.linear_stack = nn.Sequential(
            self.drop_out,
            self.l1,
        )
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        model_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, output_hidden_states=True)
        
        logits = self.linear_stack(model_output.last_hidden_state)
        
        start_logits, end_logits = logits.split(1, dim=-1)
        
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [7]:
bert_model = BertModel.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
model = QAModel()

# train loop

In [ ]:
optim = AdamW(model.parameters(), 
              lr=wandb.config['learning_rate'], 
              weight_decay=wandb.config['optim_weight_decay'])
scheduler = ExponentialLR(optim, wandb.config['scheduler_gamma'])
total_acc = []
total_loss = []

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Working on {device}')

Working on cuda


In [ ]:
wandb.watch(model, log='all', log_freq=10)

[]

In [ ]:
model.to(device)
for epoch in range(wandb.config['epochs']):
    train_epoch(model, train_dataloader, epoch+1)
    eval_model(model, eval_dataloader)

Epoch 1 :   0%|          | 0/2064 [00:00<?, ?it/s]

 0/2064 EM:0.02 | Loss:5.95 | lr:0.0000200000
 100/2064 EM:0.31 | Loss:2.01 | lr:0.0000180000
 200/2064 EM:0.37 | Loss:1.82 | lr:0.0000180000
 300/2064 EM:0.40 | Loss:1.60 | lr:0.0000180000
 400/2064 EM:0.43 | Loss:1.34 | lr:0.0000162000
 500/2064 EM:0.44 | Loss:1.30 | lr:0.0000162000
 600/2064 EM:0.46 | Loss:1.52 | lr:0.0000162000
 700/2064 EM:0.47 | Loss:1.56 | lr:0.0000145800
 800/2064 EM:0.48 | Loss:1.38 | lr:0.0000145800
 900/2064 EM:0.49 | Loss:1.39 | lr:0.0000145800
 1000/2064 EM:0.50 | Loss:1.22 | lr:0.0000131220
 1100/2064 EM:0.51 | Loss:1.56 | lr:0.0000131220
 1200/2064 EM:0.51 | Loss:1.22 | lr:0.0000131220
 1300/2064 EM:0.52 | Loss:1.56 | lr:0.0000118098
 1400/2064 EM:0.52 | Loss:1.22 | lr:0.0000118098
 1500/2064 EM:0.53 | Loss:1.37 | lr:0.0000118098
 1600/2064 EM:0.54 | Loss:1.30 | lr:0.0000106288
 1700/2064 EM:0.54 | Loss:1.02 | lr:0.0000106288
 1800/2064 EM:0.54 | Loss:1.18 | lr:0.0000106288
 1900/2064 EM:0.55 | Loss:1.18 | lr:0.0000095659
 2000/2064 EM:0.55 | Loss:1.12 |

  0%|          | 0/191 [00:00<?, ?it/s]

Post-processing 11873 example predictions split into 12199 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

...

 0/2064 EM:0.67 | Loss:1.04 | lr:0.0000045754
 100/2064 EM:0.72 | Loss:0.74 | lr:0.0000041178
 200/2064 EM:0.72 | Loss:0.72 | lr:0.0000041178
 300/2064 EM:0.72 | Loss:0.89 | lr:0.0000041178
 400/2064 EM:0.72 | Loss:0.83 | lr:0.0000037060
 500/2064 EM:0.72 | Loss:0.91 | lr:0.0000037060
 600/2064 EM:0.72 | Loss:0.86 | lr:0.0000037060
 700/2064 EM:0.72 | Loss:0.85 | lr:0.0000033354
 800/2064 EM:0.72 | Loss:0.83 | lr:0.0000033354
 900/2064 EM:0.72 | Loss:0.93 | lr:0.0000033354
 1000/2064 EM:0.72 | Loss:0.63 | lr:0.0000030019
 1100/2064 EM:0.72 | Loss:0.91 | lr:0.0000030019
 1200/2064 EM:0.72 | Loss:1.01 | lr:0.0000030019
 1300/2064 EM:0.72 | Loss:1.04 | lr:0.0000027017
 1400/2064 EM:0.72 | Loss:0.68 | lr:0.0000027017
 1500/2064 EM:0.72 | Loss:1.04 | lr:0.0000027017
 1600/2064 EM:0.72 | Loss:1.04 | lr:0.0000024315
 1700/2064 EM:0.73 | Loss:0.81 | lr:0.0000024315
 1800/2064 EM:0.73 | Loss:0.82 | lr:0.0000024315
 1900/2064 EM:0.73 | Loss:1.00 | lr:0.0000021884
 2000/2064 EM:0.73 | Loss:1.07 |

  0%|          | 0/191 [00:00<?, ?it/s]

Post-processing 11873 example predictions split into 12199 features.


  0%|          | 0/11873 [00:00<?, ?it/s]

{'exact': 68.59260507032764, 'f1': 72.0451580997796, 'total': 11873, 'HasAns_exact': 68.62348178137651, 'HasAns_f1': 75.53848888641738, 'HasAns_total': 5928, 'NoAns_exact': 68.56181665264928, 'NoAns_f1': 68.56181665264928, 'NoAns_total': 5945, 'best_exact': 68.5841825991746, 'best_exact_thresh': 0.0, 'best_f1': 72.03673562862656, 'best_f1_thresh': 0.0}


# save artifacts

In [12]:
model.to('cpu')
torch.save(model.state_dict(), 'model.pt')

In [9]:
tokenizer.save_pretrained('./saved_model/')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')